# [Solution] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# TODO: Import the necessary libs
# For example:
import os

from typing import List
from dotenv import load_dotenv
import chromadb
from tavily import TavilyClient
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool

In [3]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry.
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
@tool(
    name="retrieve_game",
    description="Retrieve game information from the vector database",
)
def retrieve_game(query: str) -> List[dict]:
    """
    Retrieve game information from the vector database based on the query.

    Args:
        query (str): A question about the game industry.

    Returns:
        List[dict]: A list of dictionaries containing game information.
    """
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")

    results = collection.query(
        query_texts=[query],
        n_results=3,
        include=['documents']
    )

    # Extract relevant information from the results
    games = []
    for doc in results['documents'][0]:
        game_info = {
            "Platform": doc.split("Platform: [")[1].split("]")[0],
            "Publisher": doc.split("Publisher: ")[1].split("\n")[0].strip(),
            "Name": doc.split("Name: ")[1].split("Release Date:")[0].strip(),
            "YearOfRelease": doc.split("Release Date: (")[1].split(")")[0],
            "Description": doc.split("Description: ")[1].strip(),
            "Genre": doc.split("Genre: ")[1].split("\n")[0].strip(),
        }
        games.append(game_info)

    return games

retrieve_game("What is a game for Nintendo Switch?")  # Example usage

[{'Platform': 'Nintendo Switch',
  'Publisher': 'Nintendo',
  'Name': 'Mario Kart 8 Deluxe',
  'YearOfRelease': '2017',
  'Description': 'An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.',
  'Genre': 'Racing'},
 {'Platform': 'Wii',
  'Publisher': 'Nintendo',
  'Name': 'Wii Sports',
  'YearOfRelease': '2006',
  'Description': "A collection of sports games that utilize the Wii's motion controls, bundled with the console to showcase its capabilities.",
  'Genre': 'Sports'},
 {'Platform': 'GameCube',
  'Publisher': 'Nintendo',
  'Name': 'Super Smash Bros. Melee',
  'YearOfRelease': '2001',
  'Description': 'A crossover fighting game featuring characters from various Nintendo franchises battling it out in dynamic arenas.',
  'Genre': 'Fighting'}]

#### Evaluate Retrieval Tool

In [5]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents,
#    it will analyze the usability of the documents to respond to that question.
#    args:
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
@tool(
    name="evaluate_retrieval",
    description="Evaluate the usability of retrieved documents for answering a question",
)
def evaluate_retrieval(question: str, retrieved_docs: List[str]) -> dict:
    """
    Evaluate the usability of retrieved documents for answering a question.

    Args:
        question (str): The original question from the user.
        retrieved_docs (List[str]): The list of retrieved documents most similar to the user query.

    Returns:
        dict: A dictionary containing evaluation results with keys 'useful' and 'description'.
    """
    llm = LLM(model="gpt-4o-mini", temperature=0.3)

    prompt = (
        "Your task is to evaluate if the documents are enough to respond to the query. "
        "Give a detailed explanation, so it's possible to take an action to accept it or not.\n\n"
        f"Question: {question}\n\n"
        "Retrieved Documents:\n" + "\n".join(retrieved_docs) + "\n\n"
        "Evaluate if these documents are useful to answer the question."
    )

    response = llm.invoke(prompt)

    # Assuming response is structured as follows:
    # {
    #     "useful": True/False,
    #     "description": "Detailed explanation of the evaluation"
    # }

    return response

#### Game Web Search Tool

In [6]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry.
@tool(
    name="game_web_search",
    description="Search the web for game industry information",
)
def game_web_search(question: str, search_depth: str = "advanced") -> List[dict]:
    """
    Search the web for game industry information based on the question.

    Args:
        question (str): A question about the game industry.

    Returns:
        List[dict]: A list of dictionaries containing search results.
    """
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    results = tavily_client.search(
        query=question,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )

    # Extract relevant information from the results
    formatted_results = {
        "answer": results.get("answer", ""),
        "results": results.get("results", [])
    }

    return formatted_results

game_web_search("What is the a game for Nintendo Switch?")  # Example usage

{'answer': "Mario Kart 8 Deluxe is a popular Nintendo Switch game. It features over 40 racers and 48 tracks. It's praised for its multiplayer capabilities and extensive content.",
 'results': [{'url': 'https://www.cnet.com/tech/gaming/top-best-nintendo-switch-games/',
   'title': 'Top 20 Best Nintendo Switch Games - June 2025 | CNET',
   'content': 'Tech\n   Gaming\n   Nintendo\n\nThe 20 Best Nintendo Switch Games Right Now\n===========================================\n\nThe Nintendo Switch\'s best games include Pokemon, Zelda, Mario and Metroid games, alongside indie hits and multiplayer party games.\n\nImage 3: Headshot of Tyler Graham\n\nImage 4: Headshot of Tyler Graham\n\nTyler Graham Writer [...] Even if you\'ve never owned a Nintendo console, you\'ve still probably played Mario Kart at some point in your life. It\'s practically _the_ go-to party game, as pretty much anyone can pick up a controller and jump in. Mario Kart 8 actually originates from the Wii U era, but the game was

### Agent

In [7]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions
# Plug all Tools you developed
agent = Agent(
    model_name="gpt-4o-mini",
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    instructions=(
        "You are an AI Research Agent for the video game industry. "
        "Your tasks include answering questions using internal knowledge, "
        "searching the web when needed, maintaining conversation state, "
        "returning structured outputs, and storing useful information for future use."
    ),
)

In [8]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
result = agent.invoke("When was Pokémon Gold and Silver released?")
print(result.get_final_state()["messages"][-1].content)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Pokémon Gold and Silver were released in 1999 for the Game Boy Color.


In [9]:
# - Which one was the first 3D platformer Mario game?
result = agent.invoke("Which one was the first 3D platformer Mario game?")
print(result.get_final_state()["messages"][-1].content)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
The first 3D platformer Mario game is **Super Mario 64**, which was released in 1996 for the Nintendo 64.


In [10]:
# - Was Mortal Kombat X realeased for Playstation 5?
result = agent.invoke("Was Mortal Kombat X realeased for Playstation 5?")
print(result.get_final_state()["messages"][-1].content)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Mortal Kombat X was originally released for PlayStation 4 in 2015. While it is not a native release for PlayStation 5, it is backward compatible and can be played on the PS5. However, some features from the PS4 version may not be available on the PS5.


### (Optional) Advanced

In [11]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes
agent = Agent(
    model_name="gpt-4o-mini",
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    instructions=(
        "You are an AI Research Agent for the video game industry. "
        "Your tasks include answering questions using internal knowledge, "
        "searching the web when needed, maintaining conversation state, "
        "returning structured outputs, and storing useful information for future use."
        "The final report should includes at least the response with citation, if any"
    ),
    long_term_memory=True  # Enable long-term memory
)

In [12]:
# - Was Mortal Kombat X realeased for Playstation 5?
result = agent.invoke("Was Mortal Kombat X realeased for Playstation 5?")
print(result.get_final_state()["messages"][-1].content)

AttributeError: 'LongTermMemory' object has no attribute 'create_session'